<a href="https://colab.research.google.com/github/financieras/ai/blob/main/logistic_regression/jupyter/sigmoide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Cargar el CSV
df = pd.read_csv('dataset_normalized_lite5.csv')

# Eliminar las columnas especificadas
columns_to_drop = ['House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']
df = df.drop(columns=columns_to_drop)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Best Hand                      1277 non-null   float64
 1   Age                            1277 non-null   float64
 2   House_Gryffindor               1277 non-null   float64
 3   Herbology                      1277 non-null   float64
 4   Defense Against the Dark Arts  1277 non-null   float64
 5   Potions                        1277 non-null   float64
 6   Charms                         1277 non-null   float64
 7   Flying                         1277 non-null   float64
dtypes: float64(8)
memory usage: 79.9 KB


In [4]:
import numpy as np

# 1. Separar variable objetivo (y) de características (X)
y = df['House_Gryffindor']

# 2. Seleccionar las características (excluyendo House_Gryffindor)
X = df[['Best Hand', 'Age', 'Herbology', 'Defense Against the Dark Arts',
        'Potions', 'Charms', 'Flying']]

# 3. Agregar columna de 1's para el término de sesgo (bias)
X = np.c_[np.ones(len(X)), X]

# Convertir a arrays de numpy para operaciones más eficientes
X = np.array(X)
y = np.array(y)

# Las dimensiones deberían ser:
# X: (1277, 8) - 1277 muestras, 8 características (incluyendo el bias)
# y: (1277,) - 1277 etiquetas

In [5]:
# Comprobar dimensiones
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

# También puedes ver las primeras filas para verificar que la estructura es correcta
print("\nPrimeras 3 filas de X (mostrando el término de bias en la primera columna):")
print(X[:3])

print("\nPrimeros 3 valores de y:")
print(y[:3])

Dimensiones de X: (1277, 8)
Dimensiones de y: (1277,)

Primeras 3 filas de X (mostrando el término de bias en la primera columna):
[[ 1.          0.         -0.63083413  0.86662781  1.0214988  -0.7028287
   1.19791313 -0.50609606]
 [ 1.          1.         -0.30779955 -1.37601754  1.14449275  0.41221315
  -1.01037058 -1.3935865 ]
 [ 1.          0.         -0.34625605  1.24909314  0.78804302  0.88932382
   1.81875448  0.08149725]]

Primeros 3 valores de y:
[0. 0. 0.]


vamos a implementar la función sigmoide. Esta función es crucial en la regresión logística ya que transforma cualquier número real en un valor entre 0 y 1, que podemos interpretar como una probabilidad.
python

In [6]:
def sigmoid(z):
    """
    Calcula la función sigmoide: σ(z) = 1/(1 + e^(-z))

    Parámetros:
    z: puede ser un número real, vector o matriz

    Retorna:
    Valor de la función sigmoide
    """
    # Usamos np.clip para evitar desbordamiento numérico
    # Limitamos los valores a [-250, 250] para evitar warnings de overflow
    z_safe = np.clip(z, -250, 250)
    return 1.0 / (1.0 + np.exp(-z_safe))

# Podemos probar la función con algunos valores para verificar que funciona correctamente
test_values = np.array([-10, -1, 0, 1, 10])
print("Valores de prueba:", test_values)
print("Valores sigmoide:", sigmoid(test_values))

Valores de prueba: [-10  -1   0   1  10]
Valores sigmoide: [4.53978687e-05 2.68941421e-01 5.00000000e-01 7.31058579e-01
 9.99954602e-01]


## Función de pérdida
Vamos a implementar la función de pérdida logarítmica (log loss) para regresión logística.
Para cada observación, la función de pérdida es:

Si y = 1: -log(h(x))
Si y = 0: -log(1 - h(x))

Donde h(x) es nuestra predicción (la salida de la función sigmoide).
Esto se puede escribir de forma compacta para todo el conjunto de datos como:
J(θ) = -(1/m) * Σ [y * log(h(x)) + (1-y) * log(1-h(x))]
Donde:

m es el número de observaciones (1277 en nuestro caso)
y son los valores reales
h(x) son las predicciones (después de aplicar la sigmoide)
Σ representa la suma sobre todas las observaciones

In [7]:
def compute_cost(X, y, theta):
    """
    Calcula la función de pérdida logarítmica

    Parámetros:
    X: matriz de características (incluyendo columna de 1's)
    y: vector de etiquetas reales
    theta: vector de parámetros

    Retorna:
    J: valor de la función de pérdida
    """
    m = len(y)

    # Calcular predicciones
    z = np.dot(X, theta)
    h = sigmoid(z)

    # Calcular pérdida logarítmica
    # Añadimos un pequeño valor epsilon para evitar log(0)
    epsilon = 1e-15
    J = -(1/m) * np.sum(y * np.log(h + epsilon) + (1-y) * np.log(1 - h + epsilon))

    return J

Esta implementación:

Calcula z = X·θ
Aplica la función sigmoide para obtener h(x)
Calcula la pérdida logarítmica
Incluye un pequeño valor epsilon para evitar problemas numéricos con log(0)